In [1]:
import pandas as pd
import janitor
import warnings
from typing import List, Dict
from IPython.display import display
import sidetable

warnings.filterwarnings("ignore")

from utilities import process_bl_json_files, pandas_to_tex

FP_US_DOMAINS = "../data/us_gov_domain_list.csv"
FP_US_BL = "../data/us_blacklight_json"
FP_IN_DOMAINS = "../data/in_gov_domain_list.csv"
FP_IN_BL = "../data/in_blacklight_json"
FP_GDIR_DOMAINS = "../data/combined_tsv_data.tsv"
FP_GDIR_BL = "../data/blacklight_json/"
FP_CC_DOMAINS = "../data/common_crawl_sample.csv"
FP_CC_BL = "../data/cc_blacklight_json"

In [2]:
bl_labels = {
    "ddg_join_ads": "Ad Trackers",
    "third_party_cookies": "Third-Party Cookies",
    "fb_pixel": "Facebook Pixel",
    "google_analytics": "Google Analytics",
    "key_logging": "Keylogging",
    "session_recording": "Session Recording",
    "canvas_fingerprinting": "Canvas Fingerprinting",
}
bl_measures = list(bl_labels.keys())
bl_measures_al1 = ["".join([var, "_al1"]) for var in bl_measures]
bl_measures_al2 = ["".join([var, "_al2"]) for var in bl_measures]
bl_measures

['ddg_join_ads',
 'third_party_cookies',
 'fb_pixel',
 'google_analytics',
 'key_logging',
 'session_recording',
 'canvas_fingerprinting']

In [3]:
def summarize_bl(
    df,
    bl_measures: List[str] = bl_measures,
    bl_measures_al1: List[str] = bl_measures_al1,
    bl_measures_al2: List[str] = bl_measures_al2,
    bl_labels: Dict[str, str] = bl_labels,
    percentiles: List[float] = [0.25, 0.5, 0.75, 0.9],
    float_format: str = "{:,.1f}",
) -> pd.DataFrame:
    """
    Summarize tracking metrics with descriptive statistics and prevalence indicators.

    Parameters
    ----------
    df : DataFrame-like
        The input DataFrame containing tracking-related columns.
    bl_measures : list of str
        Column names for the primary tracking measures (e.g., cookies, ads).
    bl_measures_al1 : list of str
        Columns indicating whether each measure was present at least once (binary indicators).
    bl_measures_al2 : list of str
        Columns indicating whether each measure was present at least twice.
    bl_labels : dict of str to str
        Dictionary mapping original variable names to display-friendly labels.
    percentiles : list of float, optional
        Percentiles to include in summary statistics (default is [0.25, 0.5, 0.75, 0.9]).
    float_format : str, optional
        Format string to apply to numeric summary values (default is "{:,.1f}").

    Returns
    -------
    pd.DataFrame
        A tidy DataFrame with transposed summary stats, and columns for share of domains
        with ≥1 and ≥2 hits per measure. Variable names are labeled according to `bl_labels`.
    """
    return (
        df.select_columns(bl_measures)
        .describe(percentiles=percentiles)
        .T.reset_index(names="var")
        .remove_columns("count")
        .applymap(
            lambda x: float_format.format(x) if isinstance(x, (float, int)) else x
        )
        .merge(
            df.select_columns(bl_measures_al1)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast1")
            .rename_column("index", "var")
            .replace("_al1", "", regex=True),
            how="left",
            on="var",
            validate="1:1",
        )
        .merge(
            df.select_columns(bl_measures_al2)
            .mean()
            .multiply(100)
            .round(1)
            .astype(str)
            .add("%")
            .reset_index(name="atleast2")
            .rename_column("index", "var")
            .replace("_al2", "", regex=True),
            how="left",
            on="var",
            validate="1:1",
        )
        .assign(var=lambda df_: df_["var"].replace(bl_labels))
    )

## US

In [4]:
df_us = pd.read_csv(FP_US_DOMAINS).clean_names().rename_column("domain_name", "domain")
df_us.head(3)

,domain,domain_type,agency,organization_name,city,state,security_contact_email
0,36thdistrictcourtmi.gov,City,Non-Federal Agency,36th District Court,Detroit,MI,govdnssecurity@36thdistrictcourt.org
1,59dcmi.gov,City,Non-Federal Agency,59th District Court,Walker,MI,soc@walker.city
2,abbevillecitymarshal.gov,City,Non-Federal Agency,Abbeville City Marshal Office,Abbeville,LA,(blank)


In [5]:
df_us_bl = (
    pd.DataFrame(process_bl_json_files(FP_US_BL))
    .merge(df_us, on="domain", how="left", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)

df_us_bl

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,domain_type,agency,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,fonda-fultonvillewastewater.gov,0,0,0,0,0,0,0,Special district,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
1,johnsoncountyso-ne.gov,0,0,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
2,scatdhhs.gov,2,8,0,0,0,0,0,Tribal,Non-Federal Agency,...,False,False,False,True,True,False,False,False,False,False
3,jeffersontown-ny.gov,0,0,0,0,0,0,0,City,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
4,floridasspeechaudiology.gov,1,0,0,0,0,0,0,State or territory,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7785,ehpsmt.gov,2,1,0,0,0,0,0,School district,Non-Federal Agency,...,False,False,False,True,False,False,False,False,False,False
7786,coopercohealth.gov,3,0,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,True,False,False,False,False,False,False
7787,randolphcountyil.gov,1,4,0,0,0,0,0,County,Non-Federal Agency,...,False,False,False,False,True,False,False,False,False,False
7788,boveymn.gov,1,0,0,0,0,0,0,City,Non-Federal Agency,...,False,False,False,False,False,False,False,False,False,False


In [6]:
df_us_summ = summarize_bl(df_us_bl)
pandas_to_tex(
    df_us_summ,
    "../tables/bl_summ_us_domains",
    escape=True,
)
df_us_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,1.7,2.8,0.0,0.0,1.0,2.0,4.0,37.0,71.7%,38.7%
1,Third-Party Cookies,1.2,4.5,0.0,0.0,0.0,1.0,4.0,86.0,26.0%,15.0%
2,Facebook Pixel,0.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,3.0%,0.0%
3,Google Analytics,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.7%,0.0%
4,Keylogging,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.8%,0.0%
5,Session Recording,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.5%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.6%,0.0%


In [7]:
!cat ../tables/bl_summ_us_domains.tex

Ad Trackers & 1.7 & 2.8 & 0.0 & 0.0 & 1.0 & 2.0 & 4.0 & 37.0 & 71.7\% & 38.7\% \\
Third-Party Cookies & 1.2 & 4.5 & 0.0 & 0.0 & 0.0 & 1.0 & 4.0 & 86.0 & 26.0\% & 15.0\% \\
Facebook Pixel & 0.0 & 0.2 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 3.0\% & 0.0\% \\
Google Analytics & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.7\% & 0.0\% \\
Keylogging & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.8\% & 0.0\% \\
Session Recording & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.5\% & 0.0\% \\
Canvas Fingerprinting & 0.0 & 0.1 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 1.6\% & 0.0\% \\

## IN

In [8]:
df_in = (
    pd.read_csv(FP_IN_DOMAINS)
    .rename_column("url", "domain")
    .dropna(subset=["domain"])
    .drop_duplicates("domain")
)
df_in.head(3)

,sector,name,domain
0,Agriculture & Cooperation,"State Wetland Authority, Haryana",https://swa.haryana.gov.in/
1,Agriculture & Cooperation,Krishi Vigyan Kendra Knowledge Network,https://kvk.icar.gov.in/
2,Agriculture & Cooperation,Bihar State Election Authority (BSEA),https://bsea.bihar.gov.in/


In [9]:
df_in[df_in.duplicated("domain", keep=False)].sort_values("domain")

,sector,name,domain


In [10]:
df_in_bl = (
    pd.DataFrame(process_bl_json_files(FP_IN_BL))
    .merge(df_in, on="domain", how="left", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)

df_in_bl

Error processing yoga.ayush.gov.in.json: list index out of range
Error processing wbkanyashree.gov.in.json: list index out of range
Error processing rural.assam.gov.in.json: list index out of range
Error processing www.odishaone.gov.in.json: list index out of range
Error processing fcsca.assam.gov.in.json: list index out of range
Error processing rdsdemaharashtra.dgt.gov.in.json: list index out of range
Error processing www.ocac.in.json: list index out of range
Error processing nhm.utl.gov.in.json: list index out of range
Error processing nehrutrophy.nic.in.json: list index out of range
Error processing www.hyderabadpolice.gov.in.json: list index out of range
Error processing cidwestbengal.gov.in.json: list index out of range
Error processing bcasindia.gov.in.json: list index out of range
Error processing www.nmdfc.org.json: list index out of range
Error processing geoportal.natmo.gov.in.json: list index out of range
Error processing asacs.assam.gov.in.json: list index out of range
Err

Error processing nild.nic.in.json: list index out of range
Error processing www.apmepma.gov.in.json: list index out of range
Error processing dogr.icar.gov.in.json: list index out of range
Error processing nests.tribal.gov.in.json: list index out of range
Error processing cpri.in.json: list index out of range
Error processing usof.gov.in.json: list index out of range
Error processing svnirtar.nic.in.json: list index out of range
Error processing dlrar.assam.gov.in.json: list index out of range
Error processing asbtc.assam.gov.in.json: list index out of range
Error processing www.plrs.org.in.json: list index out of range
Error processing vscrap.parivahan.gov.in_vehiclescrap_vahan_welcome.xhtml.json: list index out of range
Error processing www.nwapune.gov.in.json: list index out of range
Error processing handlooms.nic.in.json: list index out of range
Error processing nbcfdc.gov.in.json: list index out of range
Error processing fda.assam.gov.in.json: list index out of range
Error process

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,sector,name,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,brns.res.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,ayodhya.cantt.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2,ociservices.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,pngrb.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
4,www.rites.com,3,2,0,0,0,0,0,NaN,NaN,...,False,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2470,bhavishya.nic.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2471,vizagcustoms.gov.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2472,powerfoundation.org.in,1,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2473,darbhangadivision.bih.nic.in,0,0,0,0,0,0,0,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [11]:
df_in_summ = summarize_bl(df_in_bl)

pandas_to_tex(
    df_in_summ,
    "../tables/bl_summ_in_domains",
    escape=True,
)
df_in_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.7,2.1,0.0,0.0,0.0,1.0,3.0,42.0,28.5%,13.3%
1,Third-Party Cookies,0.5,3.5,0.0,0.0,0.0,0.0,0.0,84.0,8.3%,5.5%
2,Facebook Pixel,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.8%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.2%,0.0%
4,Keylogging,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.9%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.8%,0.0%


## GDir

* will have missing/uncleaned data, eg [https://www.govdirectory.org/singapore/Q1687545/](https://www.govdirectory.org/singapore/Q1687545/)

In [12]:
df_gdir = (
    pd.read_csv(FP_GDIR_DOMAINS, sep="\t")
    .clean_names()
    .rename_column("website", "domain")
    .rename_column("source_file", "country")
    .dropna(subset=["domain"])
    .drop_duplicates("domain")
    # ========================================================================
    # clean domain string
    .assign(
        domain=lambda df: df["domain"]
        .str.replace(r"^https?://", "", regex=True)
        .str.rstrip("/")
    )
    .drop_duplicates("domain")
)
df_gdir.head()

,name,govdirectory_url,type,domain,country
0,Cherkasy Oblast,https://www.govdirectory.org/ukraine/Q161808/,oblast of Ukraine,www.oblradack.gov.ua,Ukraine
1,Chernihiv Oblast,https://www.govdirectory.org/ukraine/Q167874/,oblast of Ukraine,cg.gov.ua,Ukraine
2,Chernivtsi Oblast,https://www.govdirectory.org/ukraine/Q168856/,oblast of Ukraine,bukoda.gov.ua,Ukraine
3,Dnipropetrovsk Oblast,https://www.govdirectory.org/ukraine/Q170672/,oblast of Ukraine,www.adm.dp.gov.ua,Ukraine
4,Donetsk Oblast,https://www.govdirectory.org/ukraine/Q2012050/,oblast of Ukraine,dn.gov.ua,Ukraine


In [13]:
df_gdir.stb.freq(["country"], sort_cols=True)

,country,count,percent,cumulative_count,cumulative_percent
0,Austria,2174,21.008891,2174,21.008891
1,Belgium,956,9.238500,3130,30.247391
2,Bermuda,13,0.125628,3143,30.373019
3,Cameroon,28,0.270584,3171,30.643603
4,Canada,33,0.318902,3204,30.962505
5,Czech Republic,233,2.251643,3437,33.214148
6,Denmark,124,1.198299,3561,34.412447
7,East Timor,15,0.144956,3576,34.557402
8,Finland,339,3.275995,3915,37.833398
9,Germany,262,2.531890,4177,40.365288


In [14]:
df_gdir_bl = (
    pd.DataFrame(process_bl_json_files(FP_GDIR_BL))
    # clean domain string
    .assign(domain=lambda df_: df_["domain"].str.rstrip("_"))
    .merge(df_gdir, on="domain", how="inner", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)
df_gdir_bl

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,name,govdirectory_url,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,www.arbetsformedlingen.se,0,1,0,0,0,0,0,Arbetsförmedlingen,https://www.govdirectory.org/sweden/Q3440237/,...,False,False,False,False,False,False,False,False,False,False
1,www.vaux-sur-sure.be,0,0,0,0,0,0,0,Vaux-sur-Sûre,https://www.govdirectory.org/belgium/Q713864/,...,False,False,False,False,False,False,False,False,False,False
2,www.la-tour-de-peilz.ch,1,0,0,0,0,0,0,La Tour-de-Peilz,https://www.govdirectory.org/switzerland/Q69721/,...,False,False,False,False,False,False,False,False,False,False
3,www.grafenegg.gv.at,0,0,0,0,0,0,0,Grafenegg,https://www.govdirectory.org/austria/Q665801/,...,False,False,False,False,False,False,False,False,False,False
4,www.adliswil.ch,0,0,0,0,0,0,0,Adliswil,https://www.govdirectory.org/switzerland/Q68210/,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3596,www.niederwil.ch,0,0,0,0,0,0,0,Niederwil,https://www.govdirectory.org/switzerland/Q65660/,...,False,False,False,False,False,False,False,False,False,False
3597,www.vogar.is,1,1,0,0,0,0,0,Vogar,https://www.govdirectory.org/iceland/Q3482077/,...,False,False,False,False,False,False,False,False,False,False
3598,www.communeduchenit.ch,0,0,0,0,0,0,0,Le Chenit,https://www.govdirectory.org/switzerland/Q52743/,...,False,False,False,False,False,False,False,False,False,False
3599,www.grindelwald.com,1,0,1,0,0,0,0,Grindelwald,https://www.govdirectory.org/switzerland/Q64004/,...,False,False,True,False,False,False,False,False,False,False


In [15]:
df_gdir_bl.stb.freq(["country"], sort_cols=True)

,country,count,percent,cumulative_count,cumulative_percent
0,Austria,387,10.747015,387,10.747015
1,Belgium,444,12.329908,831,23.076923
2,Bermuda,2,0.055540,833,23.132463
3,Cameroon,14,0.388781,847,23.521244
4,East Timor,6,0.166620,853,23.687864
5,Ghana,51,1.416273,904,25.104138
6,Iceland,58,1.610664,962,26.714801
7,Latvia,18,0.499861,980,27.214663
8,Malaysia,29,0.805332,1009,28.019994
9,Nepal,88,2.443766,1097,30.463760


In [16]:
df_gdir_summ = summarize_bl(df_gdir_bl)
pandas_to_tex(
    df_gdir_summ,
    "../tables/bl_summ_gdir_domains",
    escape=True,
)
df_gdir_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.4,0.8,0.0,0.0,0.0,0.0,1.0,8.0,23.2%,6.4%
1,Third-Party Cookies,0.2,1.0,0.0,0.0,0.0,0.0,0.0,15.0,8.0%,2.9%
2,Facebook Pixel,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.7%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.1%,0.0%
4,Keylogging,0.1,0.3,0.0,0.0,0.0,0.0,0.0,1.0,9.1%,0.0%
5,Session Recording,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.5%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.4%,0.0%


### By country

In [17]:
for ix, ctry in enumerate(list(df_gdir_bl["country"].unique())):
    _df_ctry = df_gdir.query(f"country=='{ctry}'")

    _df_ctry_bl = (
        pd.DataFrame(process_bl_json_files(FP_GDIR_BL))
        # clean domain string
        .assign(domain=lambda df_: df_["domain"].str.rstrip("_"))
        .merge(_df_ctry, on="domain", how="inner", validate="1:1")
        .assign(
            **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
            **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
        )
    )
    _df_summ = summarize_bl(_df_ctry_bl)
    pandas_to_tex(
        _df_summ,
        f"../tables/bl_summ_gdir_{ctry}_domains",
        escape=True,
    )
    print(f"{1+ix}: {ctry} (n = {len(_df_ctry_bl)})")
    display(_df_summ)

1: Sweden (n = 489)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.2,0.7,0.0,0.0,0.0,0.0,1.0,7.0,13.7%,4.1%
1,Third-Party Cookies,0.2,1.0,0.0,0.0,0.0,0.0,0.0,15.0,9.8%,2.7%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.2%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,2.7%,0.0%
5,Session Recording,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.6%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.6%,0.0%


2: Belgium (n = 444)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.4,0.9,0.0,0.0,0.0,1.0,1.0,7.0,26.6%,6.3%
1,Third-Party Cookies,0.2,1.1,0.0,0.0,0.0,0.0,0.0,12.0,5.2%,2.7%
2,Facebook Pixel,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,2.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,2.9%,0.0%
5,Session Recording,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.4%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.9%,0.0%


3: Switzerland (n = 1980)


...

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.4,0.5,0.0,0.0,0.0,1.0,1.0,1.0,44.4%,0.0%
1,Third-Party Cookies,0.7,1.3,0.0,0.0,0.0,1.0,1.6,4.0,33.3%,11.1%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.1,0.3,0.0,0.0,0.0,0.0,0.2,1.0,11.1%,0.0%
5,Session Recording,0.4,0.5,0.0,0.0,0.0,1.0,1.0,1.0,44.4%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


13: East Timor (n = 6)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
1,Third-Party Cookies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


14: Bermuda (n = 2)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,3.0,1.4,2.0,2.5,3.0,3.5,3.8,4.0,100.0%,100.0%
1,Third-Party Cookies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


## CC

In [18]:
pd.set_option("display.max_colwidth", 50)

In [19]:
def sanitize_url(url):
    """Convert URL to filename format using the exact sanitization function"""
    if pd.isna(url):
        return None
    return url.replace("http://", "").replace("https://", "").replace("/", "_")

In [20]:
df_cc = (
    pd.read_csv("../data/common_crawl_sample.csv")
    .rename_column("url", "domain")
    .assign(domain=lambda df_: df_["domain"].apply(sanitize_url))
    .drop_duplicates("domain")
)
df_cc

,urlkey,timestamp,domain,mime,mime-detected,status,digest,length,offset,filename,country,pattern,languages,encoding,redirect,truncated
0,"af,gov,moj)/content/files/crpd.pdf",20240714034654,moj.gov.af_Content_files_CRPD.pdf,text/html,text/html,404,GY5DQ2BNNLQPCCOAYY3FTL43EN6DI5LO,8518,13208486,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Afghanistan,*.gov.af,NaN,NaN,NaN,NaN
1,"af,gov,mfa,islamabad)/introductory-meeting-of-...",20240723080327,islamabad.mfa.gov.af_introductory-meeting-of-a...,text/html,text/html,200,4VOGNJXYP5M74XLAL7FZCLYDKLQQ77XU,19902,266134915,crawl-data/CC-MAIN-2024-30/segments/1720763518...,Afghanistan,*.gov.af,eng,UTF-8,NaN,NaN
2,"af,gov,mfa)/en/category/breaking-news",20240721095153,mfa.gov.af_en_category_breaking-news_,text/html,text/html,301,3I42H3S6NNFQ2MSVX7XZKYAYSCX5QBYJ,736,13362462,crawl-data/CC-MAIN-2024-30/segments/1720763517...,Afghanistan,*.gov.af,NaN,NaN,https://mfa.gov.af/en/category/breaking-news,NaN
3,"af,gov,mudh)/dr/%d8%a8%d8%b1%d9%86%d8%a7%d9%85...",20240719083155,mudh.gov.af_dr_%D8%A8%D8%B1%D9%86%D8%A7%D9%85%...,text/html,text/html,200,I5ZMS44OMMVMBHOCXUYUZO45BSQCR653,9659,371362517,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Afghanistan,*.gov.af,"fas,eng,pus",UTF-8,NaN,NaN
4,"af,gov,mail)/en/node/13864",20240724090703,mail.gov.af_en_node_13864,text/html,text/html,200,WWGQY3A4YWEINGJCFBJHGCW4ELBGXOTU,8188,322912422,crawl-data/CC-MAIN-2024-30/segments/1720763518...,Afghanistan,*.gov.af,"eng,fas",UTF-8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15678,"zw,gov,parlzim)/download/senate-hansard-08-jun...",20240716022352,parlzim.gov.zw_download_senate-hansard-08-june...,text/html,text/html,200,2QBIYNLEG55PRCKBNMT2CYWPQV7ID2HD,38024,379595814,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Zimbabwe,*.gov.zw,eng,UTF-8,NaN,NaN
15679,"zw,gov,zna)/",20240718012447,www.zna.gov.zw_,text/html,text/html,200,VLKDAIGFCUXK5QUUFKJO4BO7A7CGBG35,707,914375199,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Zimbabwe,*.gov.zw,eng,ISO-8859-9,NaN,NaN
15680,"zw,gov,parlzim)/download-category/national_ass...",20240714010830,parlzim.gov.zw_download-category_national_asse...,text/html,text/html,200,SOJBKNPG625I3OEVA3NADMQIJ5OK7HAH,21099,406553701,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Zimbabwe,*.gov.zw,eng,UTF-8,NaN,NaN
15681,"zw,gov,zimimmigration)/robots.txt",20240718222944,www.zimimmigration.gov.zw_robots.txt,text/html,text/html,404,X3DK5P2ZIDVIR6576V2I2U4UKPKJ7IUE,1252,477456,crawl-data/CC-MAIN-2024-30/segments/1720763514...,Zimbabwe,*.gov.zw,NaN,NaN,NaN,NaN


In [21]:
df_cc.stb.freq(["country"], sort_cols=True)

,country,count,percent,cumulative_count,cumulative_percent
0,Afghanistan,100,0.639141,100,0.639141
1,Albania,100,0.639141,200,1.278282
2,Algeria,100,0.639141,300,1.917423
3,Angola,100,0.639141,400,2.556564
4,Argentina,100,0.639141,500,3.195705
...,...,...,...,...,...
154,Venezuela,100,0.639141,15247,97.449827
155,Vietnam,100,0.639141,15347,98.088968
156,Yemen,100,0.639141,15447,98.728109
157,Zambia,99,0.632750,15546,99.360859


In [22]:
df_cc_bl = (
    pd.DataFrame(process_bl_json_files(FP_CC_BL, noisy=False))
    .merge(df_cc, on="domain", how="inner", validate="1:1")
    .assign(
        **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
        **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
    )
)

df_cc_bl

,domain,ddg_join_ads,third_party_cookies,canvas_fingerprinting,session_recording,key_logging,fb_pixel,google_analytics,urlkey,timestamp,...,key_logging_al1,session_recording_al1,canvas_fingerprinting_al1,ddg_join_ads_al2,third_party_cookies_al2,fb_pixel_al2,google_analytics_al2,key_logging_al2,session_recording_al2,canvas_fingerprinting_al2
0,v3bl.goszakup.gov.kz_ru_files_signature_downlo...,0,0,0,0,0,0,0,"kz,gov,goszakup,v3bl)/ru/files/signature/downl...",20240713190509,...,False,False,False,False,False,False,False,False,False,False
1,standarts.nism.gov.kg_ru_catalog_list_9_122,0,0,0,0,0,0,0,"kg,gov,nism,standarts)/ru/catalog/list/9/122",20240721171037,...,False,False,False,False,False,False,False,False,False,False
2,lvarhivs.gov.lv_index3.php?id=90091&kods=50000...,1,0,0,0,0,0,0,"lv,gov,lvarhivs)/index3.php?id=90091&kods=5000...",20240725074719,...,False,False,False,False,False,False,False,False,False,False
3,www.thimphu.gov.bt_index.php_archive_202205,0,0,0,0,0,0,0,"bt,gov,thimphu)/index.php/archive/202205",20240722115723,...,False,False,False,False,False,False,False,False,False,False
4,www.eap.bayern.de_adressen_behoerden_amtsgeric...,0,0,0,0,0,0,0,"de,bayern,eap)/adressen/behoerden/amtsgerichte...",20240720034759,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3877,ndmo.gov.vu_resources_downloads_41-past-disast...,0,0,0,0,0,0,0,"vu,gov,ndmo)/resources/downloads/41-past-disas...",20240716100422,...,False,False,False,False,False,False,False,False,False,False
3878,china.tmembassy.gov.tm_en_news_124918,1,0,0,0,0,0,0,"tm,gov,tmembassy,china)/en/news/124918",20240715151745,...,False,False,False,False,False,False,False,False,False,False
3879,kitgum.go.ug_dept_health-department,0,0,0,0,0,0,0,"ug,go,kitgum)/dept/health-department",20240716054957,...,False,False,False,False,False,False,False,False,False,False
3880,mofa.gov.iq_algeria_?p=5045,1,0,0,0,0,0,0,"iq,gov,mofa)/algeria?p=5045",20240720133130,...,False,False,False,False,False,False,False,False,False,False


In [23]:
df_cc_bl.stb.freq(["country"], sort_cols=False).head(50)

,country,count,percent,cumulative_count,cumulative_percent
0,Turkmenistan,88,2.266873,88,2.266873
1,Colombia,74,1.906234,162,4.173107
2,Vanuatu,70,1.803194,232,5.976301
3,Latvia,61,1.571355,293,7.547656
4,Ivory Coast,61,1.571355,354,9.119011
5,Luxembourg,58,1.494075,412,10.613086
6,Bhutan,57,1.468315,469,12.081401
7,New Caledonia,52,1.339516,521,13.420917
8,Croatia,52,1.339516,573,14.760433
9,Uzbekistan,51,1.313756,624,16.074189


In [24]:
df_cc_summ = summarize_bl(df_cc_bl)
pandas_to_tex(
    df_cc_summ,
    "../tables/bl_summ_cc_domains",
    escape=True,
)
df_cc_summ

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.9,2.5,0.0,0.0,0.0,1.0,3.0,65.0,45.8%,14.2%
1,Third-Party Cookies,0.7,5.5,0.0,0.0,0.0,0.0,1.0,166.0,13.9%,8.8%
2,Facebook Pixel,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.3%,0.0%
3,Google Analytics,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.4%,0.0%
4,Keylogging,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,1.4%,0.0%
5,Session Recording,0.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,2.4%,0.0%
6,Canvas Fingerprinting,0.0,0.1,0.0,0.0,0.0,0.0,0.0,1.0,0.9%,0.0%


### By country

In [25]:
for ix, ctry in enumerate(list(df_cc_bl["country"].unique())):
    _df_ctry = df_cc.query(f"country=='{ctry}'")

    _df_ctry_bl = (
        pd.DataFrame(process_bl_json_files(FP_CC_BL, noisy=False))
        # clean domain string
        .assign(domain=lambda df_: df_["domain"].str.rstrip("_"))
        .merge(_df_ctry, on="domain", how="inner", validate="1:1")
        .assign(
            **{f"{col}_al1": (lambda df, k=col: df[k] >= 1) for col in bl_measures},
            **{f"{col}_al2": (lambda df, k=col: df[k] >= 2) for col in bl_measures},
        )
    )
    _df_summ = summarize_bl(_df_ctry_bl)
    #     pandas_to_tex(
    #         _df_summ,
    #         f"../tables/bl_summ_cc_{ctry}_domains",
    #         escape=True,
    #     )
    print(f"{1+ix}: {ctry} (n = {len(_df_ctry_bl)})")
    display(_df_summ)

1: Kazakhstan (n = 43)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,1.4,1.6,0.0,0.0,1.0,2.0,4.0,5.0,55.8%,34.9%
1,Third-Party Cookies,0.4,1.1,0.0,0.0,0.0,0.0,1.0,5.0,18.6%,4.7%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.5,0.5,0.0,0.0,1.0,1.0,1.0,1.0,51.2%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


2: Kyrgyzstan (n = 39)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,1.1,0.9,0.0,0.5,1.0,1.0,3.0,3.0,74.4%,15.4%
1,Third-Party Cookies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


3: Latvia (n = 61)


...

,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
1,Third-Party Cookies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


157: Namibia (n = 1)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
1,Third-Party Cookies,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%


158: Finland (n = 1)


,var,mean,std,min,25%,50%,75%,90%,max,atleast1,atleast2
0,Ad Trackers,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
1,Third-Party Cookies,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
2,Facebook Pixel,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
3,Google Analytics,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
4,Keylogging,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
5,Session Recording,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
6,Canvas Fingerprinting,0.0,nan,0.0,0.0,0.0,0.0,0.0,0.0,0.0%,0.0%
